<a href="https://colab.research.google.com/github/matlogica/AADC-Python/blob/main/QuantLib/06-Serialization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Serialization example

This example uses results from 05-OIS-QuantLib-UserCurve.ipynb. Make sure you run it and file 05-Kernel.pkl is available.

In [ ]:
import sys
!pip install https://matlogica.com/DemoReleases/aadc-1.7.5.2401-cp3{sys.version_info.minor}-cp3{sys.version_info.minor}-linux_x86_64.whl

In [1]:
# Note that now QuantLib is not longer required
import pickle
import aadc
import numpy as np

In [2]:
with open('05-Kernel.pkl', 'rb') as f:
    Trade = pickle.load(f)

In [3]:
Trade
forecast_ratesArgs = Trade["Inputs"]["forecast_rates"]
discount_spreadArg = Trade["Inputs"]["discount_spread"]
swapNPVRes = Trade["Outputs"]["swapNPV"]
Trade["Kernel"]

In [4]:
# Same as in 05, but inputs can take arbitrary levels
forecast_rates = [    -0.004,    -0.002,    0.001,    0.005,    0.009,    0.010,    0.010,    0.012,    0.017,    0.019,    0.028,    0.032]
discount_spread = - 0.001

In [5]:
vol=0.01
num_scenarios=10000

inputs = {}
for rArg, r in zip(forecast_ratesArgs, forecast_rates):
    inputs[rArg] = float(r) * np.random.normal(1, vol, num_scenarios) 
inputs[discount_spreadArg] = discount_spread * np.random.normal(1, vol, num_scenarios)

In [6]:
request = {swapNPVRes : forecast_ratesArgs + [ discount_spreadArg ] }

In [7]:
Res = aadc.evaluate(Trade["Kernel"], request, inputs, aadc.ThreadPool(4))

In [8]:
# Now you can compare results to 05
print("Result", Res[0][Trade["Outputs"]["swapNPV"]])

Result [0.07721706 0.07770874 0.07877239 ... 0.07865259 0.07845713 0.07813713]


In [9]:
# Bucketed sensitivities of swapNPV to the zero rates
print("Forecast rates risk : ")
for rArg in forecast_ratesArgs:
    print("dNPV/dR", np.average(Res[1][Trade["Outputs"]["swapNPV"]][rArg]))

Forecast rates risk : 
dNPV/dR -0.29026696203909313
dNPV/dR -0.28541210487640556
dNPV/dR 0.0
dNPV/dR 0.0
dNPV/dR -0.010100690128293354
dNPV/dR -0.00011222989031437012
dNPV/dR -0.02715688800526541
dNPV/dR -0.07099169252177111
dNPV/dR 5.3214903218986045
dNPV/dR 0.005865765038920842
dNPV/dR 0.0
dNPV/dR 0.0
